# Read input

In [ ]:
input00 = open("input00.txt").readlines()

In [ ]:
#print(input00)
numTrainingData, numFeatures = (int(s) for s in input00[0].split())

# Create numpy arrays to store training features and labels

In [ ]:
import numpy as np
trainingFeatures = np.zeros(shape=(numTrainingData, numFeatures))
trainingLabels = np.zeros(shape=(numTrainingData,), dtype=int)

# Extract features from input dataset

In [ ]:
inputFeatures = input00[1:numTrainingData+1]

In [ ]:
len(inputFeatures)

# Clean training dataset, remove unnecessary characters and store in arrays

In [ ]:
for entryNum in range(trainingFeatures.shape[0]):
    entry = inputFeatures[entryNum].split()
    #print(entry)
    trainingLabels[entryNum] = float(entry[1])
    params = [entry[i] for i in range(2, numFeatures + 2)]
    cleanedParams = [param[param.index(":") + 1:] for param in params]
    trainingFeatures[entryNum] = np.array([float(param) for param in cleanedParams])

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
#trainingFeatures = trainingFeatures[:,[6,7,13]]
trainingFeatures = X_scaler.fit_transform(trainingFeatures)
trainingFeatures.shape

# Exract test features

In [ ]:
numTestData = int(input00[numTrainingData+1])
print(numTestData)

In [ ]:
testFeatures = np.zeros(shape=(numTestData, numFeatures))
testNames = []

In [ ]:
inputTestFeatures = input00[numTrainingData+2:numTrainingData+2+numTestData]
#print(inputTestFeatures)

In [ ]:
len(inputTestFeatures)

In [ ]:
#trainingLabels = convert_to_onehot(trainingLabels,2)
trainingLabels = np.array([i + 1 if i < 0 else i for i in trainingLabels])


# Clean test dataset, remove unnecessary characters and store in arrays

In [ ]:
for entryNum in range(testFeatures.shape[0]):
    entry = inputTestFeatures[entryNum].split()
    testNames.append(entry[0])
    params = [entry[i] for i in range(1, numFeatures + 1)]
    cleanedParams = [param[param.index(":") + 1:] for param in params]
    testFeatures[entryNum] = np.array([float(param) for param in cleanedParams])

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
#testFeatures = testFeatures[:,[6,7,13]] 
testFeatures = X_scaler.fit_transform(testFeatures)
testFeatures.shape

In [ ]:
def convert_to_onehot(labels, num_classes):
    labels = labels.reshape(-1)
    return np.eye(num_classes, dtype=int)[labels]

In [ ]:
def batch(features, labels, idx, batch_size):
    idx = idx % (features.shape[0] // batch_size)
    if (idx+1) * batch_size < labels.shape[0]:
        return features[idx * batch_size:(idx+1) * batch_size], labels[idx * batch_size:(idx+1) * batch_size]
    else:
        return features[idx * batch_size:], labels[idx * batch_size:]

# Read labels for test dataset

In [ ]:
content = open("output00.txt").readlines()
content = [line.strip('\n') for line in content]
cleanedContent = [entry[entry.index(" ") + 1:] for entry in content]

testLabels = np.array([float(s) for s in cleanedContent], dtype=int)
testLabels = np.array([i + 1 if i < 0 else i for i in testLabels])


In [ ]:
trainingFeatures[:,1].T

In [ ]:
type(trainingFeatures[0,0])

In [ ]:
testLabels.shape

# Build Neural Network

In [ ]:
import tensorflow as tf

learning_rate = 0.005

num_labels = 2

hidden_nodes_layer1 = 128

hidden_nodes_layer2 = 64

hidden_nodes_layer3 = 32

dropout_keep_prob = tf.placeholder(tf.float64, name='dropout_keep_prob')

X = tf.placeholder(tf.float64, [None, trainingFeatures.shape[1]], name='X')
y_ = tf.placeholder(tf.int32, [None], name='y_')

W_hidden1 = tf.Variable(tf.random_normal([trainingFeatures.shape[1], hidden_nodes_layer1], dtype=tf.float64), name='W_hidden1')
b_hidden1 = tf.Variable(tf.random_normal([hidden_nodes_layer1], dtype=tf.float64), name='b_hidden1')

hidden1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden1), b_hidden1))


W_hidden2 = tf.Variable(tf.random_normal([hidden_nodes_layer1, hidden_nodes_layer2], dtype=tf.float64), name='W_hidden2')
b_hidden2 = tf.Variable(tf.random_normal([hidden_nodes_layer2], dtype=tf.float64), name='b_hidden2')

hidden2 = tf.nn.relu(tf.add(tf.matmul(hidden1, W_hidden2), b_hidden2))

W_hidden3 = tf.Variable(tf.random_normal([hidden_nodes_layer2, hidden_nodes_layer3], dtype=tf.float64), name='W_hidden3')
b_hidden3 = tf.Variable(tf.random_normal([hidden_nodes_layer3], dtype=tf.float64), name='b_hidden3')

hidden3 = tf.nn.relu(tf.add(tf.matmul(hidden2, W_hidden3), b_hidden3))

dropout = tf.nn.dropout(hidden3, keep_prob=dropout_keep_prob)

W_out = tf.Variable(tf.random_normal([hidden_nodes_layer3, num_labels], dtype=tf.float64), name='W_out')
b_out = tf.Variable(tf.random_normal([num_labels], dtype=tf.float64), name='b_out')

y = tf.add(tf.matmul(dropout, W_out), b_out)

correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y),1), tf.argmax(y_,0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64), name='accuracy')
tf.summary.scalar('accuracy', accuracy)

cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=y_))
tf.summary.scalar('cross_entropy', cross_entropy)
merged = tf.summary.merge_all()
train_step = tf.train.AdamOptimizer(learning_rate, epsilon=1e-8, beta1=0.9, beta2=0.999).minimize(cross_entropy)

In [ ]:
log_dir = './logs'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver()
    
    checkpoint_file = tf.train.latest_checkpoint('checkpoint/')
    if checkpoint_file is not None:
        saver.restore(sess, checkpoint)

    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test', sess.graph)

    for i in range(10000000):
        _, loss, train_summary = sess.run([train_step, cross_entropy, merged], feed_dict={X: trainingFeatures, y_:trainingLabels, dropout_keep_prob:0.3})
        train_writer.add_summary(train_summary, i)
        if i%100 == 0:
            saver.save(sess, save_path='checkpoint/lin_classifier', global_step=i)
            print('step {}:{}'.format(i, loss))
        if i%100 == 0:
            accuracy_out, test_loss, test_summary = sess.run([accuracy,cross_entropy, merged], feed_dict={X: testFeatures, y_:testLabels, dropout_keep_prob:1.0})
            test_writer.add_summary(test_summary, i)
            print('\nTest loss:', test_loss)
            print('Accuracy:', accuracy_out)